### Import packages

In [106]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import locator as loc
import folium
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim

### Load data

In [39]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'lxml')

### Quick view and examination

In [40]:
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"XoufggpAMM8AAvvJOwcAAABL","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":949497198,"wgRevisionId":949497198,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

In [41]:
wikitable = soup.find('table')

In [42]:
wikitable.find_all('th')

[<th>Postal code
 </th>, <th>Borough
 </th>, <th>Neighborhood
 </th>]

In [43]:
for i in wikitable.find_all('th'):
    print(i.text.strip())

Postal code
Borough
Neighborhood


### Created columns and data

In [44]:
columns = [i.text.strip() for i in wikitable.find_all('th')]

In [45]:
data = [[j.text.strip() for j in i.find_all('td')] for i in wikitable.find_all("tr")[1:]]

In [46]:
trtdata = pd.DataFrame(data, columns = columns)

In [47]:
trtdata['Neighborhood'] = trtdata.Neighborhood.apply(lambda x: x.replace(' /',','))

In [59]:
trtdata.columns = ['PostalCode','Borough', 'Neighborhood']

In [60]:
trtdata.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Step 1 Remove 'Not Assigned'

In [62]:
trtdata1 = trtdata.loc[trtdata.Borough != 'Not assigned'].reset_index(drop=True)

### Step 2 Fill Neighborhood with Borough

In [63]:
trtdata1.loc[trtdata1.Neighborhood == 'Not assigned', 'Neighborhood'] = trtdata['Borough']

In [64]:
trtdata1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Step 3 Merge neighborhood

In [65]:
trtdata2 = trtdata1.groupby(['PostalCode', 'Borough']).agg(', '.join).reset_index()

In [66]:
trtdata2.shape

(103, 3)

## Adding Latitude and Longitude

In [67]:
trtdata3 = loc.locator(trtdata2, 'PostalCode', 'ca')

In [78]:
trtdata3 = trtdata3.dropna()

## Visualization in Map

In [80]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[trtdata3.lat.mean(), trtdata3.lng.mean()], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(trtdata3['lat'], trtdata3['lng'], trtdata3['Borough'], trtdata3['Neighborhood']):
    labeltext = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(labeltext, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [86]:
trt = trtdata3[trtdata3.Borough.str.contains('Toronto')].reset_index(drop=True)

In [87]:
trt.head()

,PostalCode,Borough,Neighborhood,lat,lng
0,M4E,East Toronto,The Beaches,43.6784,-79.2941
1,M4K,East Toronto,"The Danforth West, Riverdale",43.6803,-79.3538
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.6693,-79.3155
3,M4M,East Toronto,Studio District,43.6561,-79.3406
4,M4N,Central Toronto,Lawrence Park,43.7301,-79.3935


In [100]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [101]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, labeltext in zip(trt['lat'], trt['lng'], trt['Neighborhood']):
    label = folium.Popup(labeltext, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [105]:
# add markers to map
for row in trt.itertuples():
    map_toronto.add_child(folium.Marker(location=[row.lat,row.lng],
           popup=row.Neighborhood))

map_toronto

In [114]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

mc = MarkerCluster()

# add markers to map
for row in trt.itertuples():
    mc.add_child(folium.Marker(location=[row.lat,row.lng],
           popup=row.Neighborhood))

    
map_toronto.add_child(mc)
map_toronto